In [2]:
# General python libraires 
import os

# General DS libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

# Parallelized libraries 
import dask.dataframe as dd 

In [3]:
# Getting files 
path = 'C:/Users/redha.cherif_artefac/GitHub_perso/American_express_comp/input/'
train = dd.read_csv(os.path.join(path, 'train_data.csv'))
train_labels = dd.read_csv(os.path.join(path, 'train_labels.csv'))
test = dd.read_csv(os.path.join(path, 'test_data.csv'))

In [18]:
print(len(train))
print(len(test))

5531451
11363762


In [45]:
print(dd.compute(train.S_2.min(),train.S_2.max()))
print(dd.compute(test.S_2.min(), test.S_2.max()))
print('train: 12 months')
print('test: 18 months')

('2017-03-01', '2018-03-31')
('2018-04-01', '2019-10-31')
train: 12 months
test: 18 months


In [15]:
train_cust_ID = train.customer_ID.unique().compute()
train_labels_custID = train_labels.customer_ID.unique().compute()
test_cust_ID = test.customer_ID.unique().compute()

In [21]:
print(f'Number of customers in train : {len(train_cust_ID)}')
print(f'Number of customers in test : {len(test_cust_ID)}')

Number of customers in train : 458913
Number of customers in test : 924621


In [53]:
train_test_join = pd.merge(train_cust_ID,test_cust_ID,how='inner')
train_trainLabel_join = pd.merge(train_cust_ID,train_labels_custID,how='inner')
print(f'Number of common customers in the train and the test : {len(train_test_join)}')
print(f'Number of common customers in the train and the train_labels : {len(train_trainLabel_join)}')
print('')
print('Therefore, all customers in the test are different from the ones in the train ! No leakage possible at first glance.')
print('The customers in the train_labels are the same than in the train')

Number of common customers in the train and the test : 0
Number of common customers in the train and the train_labels : 458913

Therefore, all customers in the test are different from the ones in the train ! No leakage possible at first glance.
The customers in the train_labels are the same than in the train


In [69]:
cat_feats = train[['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]
cat_feats.head()

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68
0,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
1,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
2,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
3,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
4,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0


In [83]:
print(f'Unique values for B_30: {cat_feats.B_30.unique().compute()}')
print(f'Unique values for B_38: {cat_feats.B_38.unique().compute()}')
print(f'Unique values for D_114: {cat_feats.D_114.unique().compute()}')
print(f'Unique values for D_116: {cat_feats.D_116.unique().compute()}')
print(f'Unique values for D_117: {cat_feats.D_117.unique().compute()}')
print(f'Unique values for D_120: {cat_feats.D_120.unique().compute()}')
print(f'Unique values for D_126: {cat_feats.D_126.unique().compute()}')
print(f'Unique values for D_63: {cat_feats.D_63.unique().compute()}')
print(f'Unique values for D_64: {cat_feats.D_64.unique().compute()}')
print(f'Unique values for D_66: {cat_feats.D_66.unique().compute()}')
print(f'Unique values for D_68: {cat_feats.D_68.unique().compute()}')

Unique values for B_30: 0    0.0
1    2.0
2    1.0
3    NaN
Name: B_30, dtype: float64
Unique values for B_38: 0    2.0
1    1.0
2    3.0
3    5.0
4    6.0
5    7.0
6    4.0
7    NaN
Name: B_38, dtype: float64
Unique values for D_114: 0    1.0
1    0.0
2    NaN
Name: D_114, dtype: float64
Unique values for D_116: 0    0.0
1    NaN
2    1.0
Name: D_116, dtype: float64
Unique values for D_117: 0    4.0
1   -1.0
2    6.0
3    2.0
4    1.0
5    NaN
6    3.0
7    5.0
Name: D_117, dtype: float64
Unique values for D_120: 0    0.0
1    1.0
2    NaN
Name: D_120, dtype: float64
Unique values for D_126: 0    1.0
1    NaN
2    0.0
3   -1.0
Name: D_126, dtype: float64
Unique values for D_63: 0    CR
1    CO
2    CL
3    XZ
4    XM
5    XL
Name: D_63, dtype: object
Unique values for D_64: 0      O
1      R
2    NaN
3      U
4     -1
Name: D_64, dtype: object
Unique values for D_66: 0    NaN
1    1.0
2    0.0
Name: D_66, dtype: float64
Unique values for D_68: 0    6.0
1    2.0
2    3.0
3    NaN
4    

In [3]:
print('The number of S columns : 22 \n (S_2, S_3, S_5, S_6, S_7, S_8, S_9, S_11, S_12, S_13, S_15, S_16, S_17, S_18, S_19, S_20, S_22, S_23, S_24, S_25, S_26, S_27')

The number of S columns : 22 
 (S_2, S_3, S_5, S_6, S_7, S_8, S_9, S_11, S_12, S_13, S_15, S_16, S_17, S_18, S_19, S_20, S_22, S_23, S_24, S_25, S_26, S_27
